# Bandpass Filtering In Audio Front End (AFE) Of Xylo™Audio 3

 In this tutorial we will review main features of the filterbank in Xylo™Audio 3 and provide instructions to reconfigure them if needed. 

 The key feature extraction module in Xylo™Audio 3 is the collectuion of Mel-filterbanks, which compute the time-frequency transform of the input audio signal. Central frequency and all parameters of the filterbank have 

 been designed for efficient audio processing, convering the range of $100~Hz$ up to around $17~KHz$ audio spectrum. They also have been hard coded in AFESim3 (:py:class:`~.syns65302.AFESim`).  

  Main features include:

 * Logarithmically spaced Mel-filterbanks  matched to human perception mechanism, with following **frequency scaling** $\alpha$ and **quality factor** $Q$:

     $ \alpha = (\frac{17000}{100})^{1/(16-1)}= 170^{\frac{1}{15}} = 1.4083$    and $~~~~~~ Q \approx 6$ 
     
 *  Implemented with order-1 Butterworth filters  with digital transfer function: $~~~~~~H(z)= \frac{b(z)}{a(z)}=\frac{b_0 + b_1 z + b_2 z^2}{a_0 + a_1 z + a_2 z^2}$.
 
 *   Implemented as a cascaded of AR (auto-regressive filter) $H_1(z)=\frac{1}{a(z)}$ followed by the MA (moving average) filter $H_2(z)=b(z)$, as illustrated in the  block diagram below

 *   **In Rockpool:**  implemented as quantized digital filterbank with hard coded parameters. (See :py:class:`~.syns65302.AFESim`). 


Default central frequencies in Xylo™Audio 3 are:

In [ ]:
import numpy as np
f0 = 100
f15 = 17000
N_filters = 16 # number of filters
alpha = np.pow(f15/f0, 1/(N_filters-1))
freqs = [f0]
for i in range(1,N_filters):
    freqs.append(int(alpha*freqs[-1]))
print(f'designed filter centers are: {freqs}')    

Defining AFESim:

In [ ]:
from rockpool.devices.xylo.syns65302 import AFESimExternal

dt_s = 0.009994
afesim_external = AFESimExternal.from_specification(spike_gen_mode="divisive_norm",
                                                    fixed_threshold_vec = None,
                                                    rate_scale_factor=63,
                                                    low_pass_averaging_window=84e-3,
                                                    dn_EPS=32,
                                                    dt=dt_s,)

Let's demonstrate the response of filters in the simulator :py:class:`~.syns65302.AFESim` by passing monotone sinusoids as input (first and fifth filters with central frequencies $100~Hz$ and $390~Hz$). 

In [ ]:
from rockpool.devices.xylo.syns65302.afe import ChipButterworth
import matplotlib.pyplot as plt

test_freqs = [freqs[0], freqs[4]]  #input frequency 
fs = 48000 #sampling rate 
N = 5 #input duration (seconds)
time = np.linspace(0,N,int(N*fs))

EPS = 0.00001
fb = ChipButterworth()
B_in = fb.bd_list[0].B_in + 4 # quantization range for input signal

plt.figure(figsize=(11,4))
for i,f in enumerate(test_freqs):
    sig_in = np.sin(2*np.pi*f*time)
    # quantize the sinal
    sig_in = sig_in/np.max(np.abs(sig_in)) * (1 + EPS) * 2**(B_in-1)
    q_sig_in = sig_in.astype(np.int64)
    output,_, _ = afesim_external((q_sig_in,fs))
    ax = plt.subplot(1,2,i+1)
    plt.imshow(output.T, aspect='auto'); 
    plt.grid(True); plt.xlabel('Time(sec)'); plt.ylabel('Freq(Hz)')
    ax.set_xticks(range(0,500, 100)); ax.set_yticks(range(N_filters))
    ax.set_xticklabels( [int(t * np.round(dt_s, decimals=2)) for t in ax.get_xticks()])
    ax.set_yticklabels(freqs); plt.colorbar()

### Reconfiguration of filter parameters


In [ ]:
from IPython.display import Image
Image("figures/block_diagram.png")

As mentioned above, this design value of parameters are suitable for efficient audio processing. They have been carefully chosen for good coverage of the the range between $100~Hz$ and $17~KHz$ and numerical stability. 

Changing these parameters is not recomeneded for audio applications. 

However, coresponding registers in the devkit are configurable and users with customized use cases can modify them and shift the center of filters to desired values.  

Main parameters to modify are $a_1$ and $a_2$ in $H(z)$ and using $scipy.signal.iirpeak()$  we can find  the parameters of the transfer function. 

Following block can be used to calculate transfer function of a desired filter  (required $a_1$ and $a_2$) parameters. 


In [5]:
from scipy import signal
# Given parameters
f0 = 100        # Center frequency (Hz)
Q = 6           # Q factor
fs = 48000      # Sampling rate (Hz)
# Get digital filter coefficients (z-domain)
b, a = signal.iirpeak(f0, Q, fs=fs)

As in :py:class:`~.syns65302.AFESim` quantized version of filters are implemented (to match Xylo™Audio 3), $a_1$ and $a_2$ parameters need to be quantized as follows:


- $$\tilde{a}_1=[2^{B_b} 2^{B_{a,f}} a_1]~~~~~~~~~\tilde{a}_2=[2^{B_b} 2^{B_{a,f}} a_2]$$
where: 
- $B_b$ : bits needed for scaling b0
- $B_{af}$ : bits needed for encoding the fractional parts of taps

Without going into details, $B_b$ and $B_{af}$ are integer values defining precision of each parameter in the filter. They have been hard coded in :py:class:`~.syns65302.AFESim` for each of 16 filters and should not be modified. 

Please note that central frequency of low-frequency filters in HDK can shift from design value due to poor dynamic range in their filter coefficients and with chosen values (around 16 bits).

While the consequence is not significant for audio processing configuration, users need to have this in mind for the use cases that require central frequcies below $100~Hz$

To change the center of filters to a desired range one can get $a_1$ $a_2$ parameters with the following code and replace 

In [ ]:
import numpy as np
f0 = 300
f15 = 20000
N_filters = 16 # number of filters
alpha = np.pow(f15/f0, 1/(N_filters-1))
new_freqs = [f0]
for i in range(1,N_filters):
    new_freqs.append(int(alpha*new_freqs[-1]))
print(f'designed filter centers are: {new_freqs}')  

In [8]:
from scipy import signal
Q = 6  # Qfactor
new_params = []
for f in new_freqs:
    # Get digital filter coefficients (z-domain)
    filter_params = signal.iirpeak(f, Q, fs=fs)
    new_params.append(filter_params)

In [ ]:
import numpy as np

# Define the BlockDiagram class to store the filter parameters
class BlockDiagram:
    def __init__(self, B_worst_case, B_b, B_af, a1, a2, scale_out):
        self.B_worst_case = B_worst_case  # Worst case bandwidth or some parameter
        self.B_b = B_b  # Bandwidth (or other related parameter)
        self.B_af = B_af  # Filter bandwidth factor or parameter
        self.a1 = a1  # Coefficient a1 for filter transfer function
        self.a2 = a2  # Coefficient a2 for filter transfer function
        self.scale_out = scale_out  # Output scaling factor (or gain)

# Ground truth centers (in Hz)
ground_truth_centers = [105, 136, 201, 279, 390, 542, 762, 1070, 1503, 2110, 
                        2963, 4161, 5843, 8204, 11582, 16611]

# Convert Hz to Mel scale
def hz_to_mel(hz):
    return 2595 * np.log10(1 + hz / 700)

# Convert Mel scale back to Hz
def mel_to_hz(mel):
    return 700 * (10**(mel / 2595) - 1)

# Convert all ground truth Hz centers to Mel scale
mel_centers = [hz_to_mel(f) for f in ground_truth_centers]

# Calculate the Mel frequencies for filters' desired positions
# Now let's adjust the filter parameters for these centers
def design_filter_for_center(mel_center):
    # Convert Mel center to Hz
    center_hz = mel_to_hz(mel_center)
    
    # Placeholder design: you would need to adjust the parameters based on your actual filter design
    a1 = -int(center_hz)  # rough approximation
    a2 = int(center_hz / 2)  # another rough guess for filter parameter
    scale_out = 0.6  # Placeholder scaling factor
    
    return BlockDiagram(
        B_worst_case=5,
        B_b=6,
        B_af=8,
        a1=a1,
        a2=a2,
        scale_out=scale_out
    )

# Design the filters with desired centers
filters = []
for mel_center in mel_centers:
    filter_obj = design_filter_for_center(mel_center)
    filters.append(filter_obj)

# Print the desired and estimated filter centers in Hz
print("Desired and Calculated Centers (in Hz):")
for i, (ground_truth, mel_center) in enumerate(zip(ground_truth_centers, mel_centers)):
    print(f"Filter {i} - Ground Truth Center: {ground_truth} Hz, Estimated Mel Center: {mel_center:.2f} Mel, Estimated Hz Center: {mel_to_hz(mel_center):.2f} Hz")


Explanation:
- BlockDiagram class: This class stores filter parameters such as a1, a2, scale_out, and other attributes that describe each filter.
- Filter Design: For each Mel center, I am designing a filter and using a simple approximation to calculate the filter parameters (a1, a2, etc.). This is a placeholder that you would typically replace with the actual filter design formula or algorithm.

- Mel Scale: The code converts your desired center frequencies in Hz to Mel frequencies and then computes the corresponding filter parameters.
- You can replace the filter design method inside design_filter_for_center with more sophisticated methods like Butterworth or Chebyshev filter design algorithms to generate actual filter coefficients (a1, a2, etc.) based on the desired frequency response.



In [ ]:
from scipy import signal

# Given parameters
f0 = 15000       # Center frequency (Hz)
Q = 6           # Q factor
fs = 48000      # Sampling rate (Hz)

# Get digital filter coefficients (z-domain)
b, a = signal.iirpeak(f0, Q, fs=fs)

print("Numerator coefficients (b):", b)
print("Denominator coefficients (a):", a)


In [ ]:
b_b = 1
b_af = 14

a0 = 2** (b_b + b_af) 
A = a0*a
A
# a1 = a0 *a1
# a2 = [a0*a2]

In [ ]:
import numpy as np
a = np.pow(170, 1/15)
a